<a href="https://colab.research.google.com/github/royam0820/crewai/blob/main/L2_research_write_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DeepLearning Course on [Multi AI Agent Systems with crewAI](https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/4/create-agents-to-research-and-write-an-article-(code))

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`



```
crewai==0.28.8
crewai_tools==0.1.6
langchain_community==0.0.29# This is formatted as code
```



In [1]:
!pip install crewai==0.28.8
!pip install crewai_tools--0.1.6
!pip install langchain_community==0.0.29 # this is formatted as code
!pip install utils

  Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.29
    Uninstalling langchain-community-0.0.29:
      Successfully uninstalled langchain-community-0.0.29
ERROR: Could not find a version that satisfies the requirement crewai_tools--0.1.6 (from versions: none)
ERROR: No matching distribution found for crewai_tools--0.1.6
  Using cached langchain_community-0.0.29-py3-none-any.whl (1.8 MB)
  Attempting uninstall: langchain_community
    Found existing installation: langchain-community 0.0.38
    Uninstalling langchain-community-0.0.38:
      Successfully uninstalled langchain-community-0.0.38
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-community<0.1,>=0.0.38, but you have langchain-community 0.0.29 which is

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [5]:
import os
from google.colab import userdata

In [6]:
openai_api_key = userdata.get("OPENAI_API_KEY")

In [8]:
os.environ["OPENAI_API_KEY"] = openai_api_key

In [9]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [10]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	  verbose=True
)

### Agent: Writer

In [11]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [12]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [13]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [14]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [15]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [16]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan for Blog Article on Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of the latest trends in AI technology and key players in the industry

Key Points:
1. Latest Trends in AI:
- Advancements in natural language processing and speech recognition
- Increased adoption of AI in healthcare, finance, and retail sectors
- Growth of AI-powered virtual assistants and chatbots

2. Key Players in AI Industry:
- Google, Am

- Display the results of your execution as markdown in the notebook.

In [18]:
from IPython.display import Markdown
Markdown(result)

# The Evolution of Artificial Intelligence: Trends, Players, and Impact

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way businesses operate and individuals interact with technology. The latest trends in AI technology, coupled with advancements in natural language processing and speech recognition, have paved the way for increased adoption in sectors such as healthcare, finance, and retail. Companies like Google, Amazon, and Microsoft are at the forefront of AI research and development, while startups like OpenAI and UiPath are disrupting the industry with innovative solutions. Academic institutions also play a crucial role in contributing to AI research and talent development, shaping the future of this rapidly evolving field.

## Latest Trends in AI

In recent years, AI-powered virtual assistants and chatbots have become ubiquitous, enhancing customer service and streamlining processes in various industries. The continuous advancements in deep learning algorithms have led to breakthroughs in AI capabilities, enabling machines to perform complex tasks with human-like precision. Furthermore, the ethical considerations surrounding AI development and deployment have gained prominence, prompting discussions on responsible innovation and the societal impact of AI technologies. As AI continues to evolve, it is essential for businesses and policymakers to address the challenges and opportunities presented by this transformative technology.

## Key Players in AI Industry

The AI industry is dominated by tech giants like Google, Amazon, and Microsoft, who invest heavily in research and development to drive innovation in AI. These companies leverage their vast resources and expertise to push the boundaries of AI capabilities, creating cutting-edge solutions that shape the future of technology. At the same time, startups such as OpenAI and UiPath bring fresh perspectives and disruptive technologies to the market, challenging established players and driving industry-wide innovation. With academic institutions also contributing to AI research and talent development, the landscape of the AI industry is dynamic and diverse, offering a wealth of opportunities for collaboration and advancement.

## Noteworthy News in AI

Recent breakthroughs in deep learning algorithms have enabled AI systems to achieve remarkable feats, from natural language processing to image recognition. However, as AI technologies become more sophisticated, ethical considerations become increasingly important. The ethical implications of AI development, including issues of bias, privacy, and transparency, have sparked debates on responsible AI innovation and the need for regulatory frameworks to ensure the ethical deployment of AI technologies. Moreover, the impact of AI on job automation and workforce transformation raises questions about the future of work and the role of AI in shaping the global economy. By addressing these challenges and opportunities, the AI industry can pave the way for a more inclusive and sustainable future for all.

In conclusion, Artificial Intelligence continues to reshape the world as we know it, with the latest trends, key players, and noteworthy news driving innovation and transformation across industries. By staying informed and engaging with the evolving landscape of AI technology, professionals, entrepreneurs, students, and enthusiasts can harness the power of AI to drive business growth, foster responsible innovation, and shape a brighter future for all. Stay updated on AI news and developments, share your thoughts and experiences with AI technology, and explore related resources for further reading and learning. The future of AI is here, and it's up to us to shape it responsibly and ethically.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
# topic = "YOUR TOPIC HERE"
# result = crew.kickoff(inputs={"topic": topic})

In [19]:
topic = "Quantum mechanics"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Quantum mechanics.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Quantum Mechanics Blog Article

Outline:
I. Introduction
    A. Definition of Quantum Mechanics
    B. Importance and Applications in the Modern World

II. Key Points
    A. History and Development of Quantum Mechanics
    B. Fundamental Principles of Quantum Mechanics
    C. Quantum Mechanics vs. Classical Mechanics
    D. Quantum Entanglement and Superposition
    E. Quantum Computing and Future Implications

III. Call to Action
    A. Encourage further exploration and learning about Quantum Mechan

In [26]:
Markdown(result)

# Unraveling the Mysteries of Quantum Mechanics

## Introduction
Quantum mechanics, a branch of physics that deals with the behavior of particles at the smallest scales, has long been a subject of fascination and intrigue. From quantum computing to quantum teleportation, the field of quantum mechanics is constantly evolving and pushing the boundaries of what we thought was possible. In today's fast-paced world, understanding quantum mechanics is more important than ever as it underpins many cutting-edge technologies and scientific advancements.

## Key Points

### Latest Trends in Quantum Mechanics
Recent advancements in quantum computing have shown promising results, with companies like IBM and Google leading the way in developing powerful quantum machines. Quantum teleportation experiments have also captured the imagination of scientists and the public alike, showcasing the potential for instantaneous communication over long distances. Furthermore, research into quantum entanglement continues to uncover the mysterious connections between particles that defy classical physics laws.

### Key Players in Quantum Mechanics
Trailblazing scientists like Niels Bohr, Albert Einstein, and Richard Feynman have made significant contributions to the field of quantum mechanics, laying the foundation for our current understanding of the quantum world. Companies investing in quantum computing research are driving innovation and shaping the future of technology by harnessing the power of quantum mechanics.

### Noteworthy News in Quantum Mechanics
Breakthroughs in quantum cryptography are enhancing data security and privacy in the digital age, while applications of quantum mechanics in industries such as healthcare, finance, and energy are revolutionizing traditional practices. The potential for quantum technologies to transform various sectors is vast, with exciting developments on the horizon.

## Target Audience Analysis

### Interests
For science enthusiasts, quantum mechanics offers a deep dive into the fundamental principles that govern the universe, sparking curiosity and wonder. Technology professionals are drawn to the practical applications of quantum mechanics in cutting-edge technologies, seeking to stay ahead of the curve. Students studying physics or engineering are eager to grasp the complex concepts of quantum mechanics to prepare for future advancements in their respective fields.

### Pain Points
Understanding the intricacies of quantum mechanics can be a daunting task for many, as the concepts often challenge our classical understanding of the world. Keeping up with the latest developments in the field requires continuous learning and adaptation, posing a challenge for enthusiasts and professionals alike.

## Conclusion
In conclusion, quantum mechanics remains a captivating and ever-evolving field that holds immense potential for scientific discovery and technological innovation. By exploring the latest trends, key players, and noteworthy news in quantum mechanics, we can gain a deeper appreciation for the complexities of the quantum world and the impact it has on our daily lives. Stay tuned for more insights and updates on quantum mechanics, and subscribe to our newsletter to stay informed about the latest advancements in science and technology.

Remember to stay curious and keep exploring the mysteries of quantum mechanics!

*Call to Action: Stay tuned for our upcoming articles on quantum mechanics and subscribe to our newsletter for the latest updates in the world of science and technology.*

## Downloading the markdown file

In [26]:
# !pip install markdown

In [32]:
import markdown

In [34]:
markdown_content = Markdown(result)
file_name = "quantum_mechanics.md"

with open(file_name, 'w', encoding='utf-8') as file:
    file.write(str(markdown_content))


In [25]:
with open(file_name, 'r') as file:
    content = file.read()
print(content)


<IPython.core.display.Markdown object>


<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).